In [8]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds
import json
import datetime  # For datetime objects

In [9]:
# https://github.com/happydasch/btoandav20
import btoandav20 as bto

In [10]:
''' Order info '''


class St(bt.Strategy):

    def __init__(self):
        self.order = None
        self.mybuysignals = {
            "buysignal1": False,
            "buysignal2": False,
            "buysignal3": False,
            "buysignal4": True,
            "buysignal5": False,

        }

    def notify_store(self, msg, *args, **kwargs):
        if "clientExtensions" in msg:
#             print(msg)
#             j_msg = json.loads(json.dumps(msg))
#             print(j_msg)
             txt = ["*" * 5, "STORE NOTIF:", msg]
             print(", ".join(txt))
#             o_info = msg["clientExtensions"]
#             buytrigger = o_info["comment"]["buytrigger"]

    def next(self):
        if self.order:
            return

        for k, v in self.mybuysignals.items():
            if v:
                self.order = self.buy(
                    size=1,
                    buytrigger=k
                    )

In [24]:
class m5VolExpansion(bt.Strategy):

    params = (
        ('tpRR', 2),
        ('printlog', True),
    )
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0) #datetime.datetime(2023, 5, 1) #
            print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        self.order = None
        self.mybuysignals = {
            "buysignal1": False,
            "buysignal2": False,
            "buysignal3": False,
            "buysignal4": True,
            "buysignal5": False,
        }

#     def notify_store(self, msg, *args, **kwargs):
#         self.log('store: %s' % (msg))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.4f' % self.datas[0].close[0])
        
        if self.order:
            return

        for k, v in self.mybuysignals.items():
            if v:
                self.order = self.buy(
                    size=1,
                    buytrigger=k
                    )
    def stop(self):
        self.log('(TP R:R %2d) Ending Value %.2f' %
                 (self.params.tpRR, self.broker.getvalue()), doprint=True)

In [26]:
with open("./secret/config-practice.json", "r") as file:
    config = json.load(file)

storekwargs = dict(
    token=config["oanda"]["token"],
    account=config["oanda"]["account"],
    practice=config["oanda"]["practice"],
    notif_transactions=True,
    stream_timeout=10,
)
store = bto.stores.OandaV20Store(**storekwargs)
datakwargs = dict(
    historical=True,
    fromdate=datetime.datetime(2023, 5, 1),
    todate=datetime.datetime(2023, 5, 2),
    timeframe=bt.TimeFrame.Minutes,
    compression=5,
    tz='Europe/Berlin',
    backfill=False,
    backfill_start=False
)
data = store.getdata(dataname="EUR_USD", **datakwargs)
data.resample(
    timeframe=bt.TimeFrame.Minutes,
    compression=5)  # rightedge=True, boundoff=1)
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.setbroker(store.getbroker())
cerebro.addstrategy(m5VolExpansion) #St)
cerebro.run()

2023-05-01, Close, 1.1013
2023-05-01, Close, 1.1016
2023-05-01, Close, 1.1016
2023-05-01, Close, 1.1016
2023-05-01, Close, 1.1015
2023-05-01, Close, 1.1013
2023-05-01, Close, 1.1014
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1012
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1012
2023-05-01, Close, 1.1012
2023-05-01, Close, 1.1011
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1013
2023-05-01, Close, 1.1012
2023-05-01, Close, 1.1010
2023-05-01, Close, 1.1007
2023-05-01, Close, 1.1009
2023-05-01, Close, 1.1006
2023-05-01, Close, 1.1006
2023-05-01, Close, 1.1007
2023-05-01, Close, 1.1008
2023-05-01, Close, 1.1009
2023-05-01, Close, 1.1004
2023-05-01, Close, 1.1005
2023-05-01, Close, 1.1005
2023-05-01, Close, 1.1003
2023-05-01, 